# Impact of artificial snow on the albedo: Results of the AWS data on Patscherkofel

In [ ]:
from IPython.display import HTML


In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
# Necessary packages/modules to import
import pandas as pd  
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
import xlrd
### conda install plotly
### pip install cufflinks --upgrade
import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

Two automatic weather stations (AWS) were mounted in the skiing area of Patscherkofel on 23rd of March 2018 to investigate whether artificial snow has an impact on the change of reflectivity of shortwave radiation and consequently an impact on the energy budget. For this purpose two CNR4 Net Radiometer were installed, which measure the incoming and outgoing longwave(LW) and shortwave(SW) radiation. First, both were placed in natural finish terrain on snow next to a ski slope. On the 6th of April 2018 one station was moved on the ski slope. The measurement period was from 23rd of March until the 9th of May. It has to be mentioned that there was a huge amount of natural snow in this winter and we do not know how much artificial snow actually was produced, so it is hard to quantify how strong the effect actually is.

In [ ]:
# read Patscherkofel data
df1 = pd.read_table('CR6_Patscherkofel_1_Table1_Logger_15052018.dat', skiprows=[2,3], header = 1, delimiter=',', index_col=0, parse_dates=True)
df2 = pd.read_table('CR6_Patscherkofel_2_Table1_Logger_15052018.dat',skiprows=[2,3], header = 1, delimiter=',', index_col=0, parse_dates=True) 
# only those values during the measurement period
df3 = df1.loc['2018-03-24':'2018-05-08']
df4 = df2.loc['2018-03-24':'2018-05-08']

### Data
- The data df3(see code) belongs to the Patscherkofel 1 station: stays on the natural snow.
- The data df4(see code) belongs to the Patscherkofel 2 station: first on natural snow, from 6.04.2018 on on the ski slope.
- The data is examined from 24.03.2018 to 08.05.2018 to avoid errors from the installation.
- time steps: 1 minute

In [ ]:
#Convert data to numeric data:
df3.PTemp_C_Avg=pd.to_numeric(df3.PTemp_C_Avg, errors='coerce')
df3.BattV_Avg=pd.to_numeric(df3.BattV_Avg, errors='coerce')
df3.BattV_Max=pd.to_numeric(df3.BattV_Max, errors='coerce')
df3.BattV_Min=pd.to_numeric(df3.BattV_Min, errors='coerce')
df3.SWUpper_Avg=pd.to_numeric(df3.SWUpper_Avg, errors='coerce')
df3.SWLower_Avg=pd.to_numeric(df3.SWLower_Avg, errors='coerce')
df3.LWUpper_Avg=pd.to_numeric(df3.LWUpper_Avg, errors='coerce')
df3.LWLower_Avg=pd.to_numeric(df3.LWLower_Avg, errors='coerce')
df3.CNR4TC_Avg=pd.to_numeric(df3.CNR4TC_Avg, errors='coerce')
df3.CNR4TK_Avg=pd.to_numeric(df3.CNR4TK_Avg, errors='coerce')
#
df4.PTemp_C_Avg=pd.to_numeric(df4.PTemp_C_Avg, errors='coerce')
df4.BattV_Avg=pd.to_numeric(df4.BattV_Avg, errors='coerce')
df4.BattV_Max=pd.to_numeric(df4.BattV_Max, errors='coerce')
df4.BattV_Min=pd.to_numeric(df4.BattV_Min, errors='coerce')
df4.SWUpper_Avg=pd.to_numeric(df4.SWUpper_Avg, errors='coerce')
df4.SWLower_Avg=pd.to_numeric(df4.SWLower_Avg, errors='coerce')
df4.LWUpper_Avg=pd.to_numeric(df4.LWUpper_Avg, errors='coerce')
df4.LWLower_Avg=pd.to_numeric(df4.LWLower_Avg, errors='coerce')
df4.CNR4TC_Avg=pd.to_numeric(df4.CNR4TC_Avg, errors='coerce')
df4.CNR4TK_Avg=pd.to_numeric(df4.CNR4TK_Avg, errors='coerce')

# 1.  Battery

Figure 1 shows the minimum battery voltage for both stations.
In the first days problems occured with the battery of Patscherkofel 1 and 2, which can be seen by decreasing minimum voltage (at the evening of 26.03.2018). So, no data was recorded between 00:10 and 11:18 at March 27th. The issue was solved by maintenance work of Wolfgang Gurgiser on this same day.
Due to the heavy snowfall, the solar panels got covered with snow and hence the batteries could not be recharged on the day 2018-03-26. So the voltage was sinking up to the minimum of 11 V in the night and the station went out of power.

Snow might have covered Patscherkofel 1 station at March 29th, as the battery was not recharged compared to Patscherkofel 2. 

That means that also possible snow cover over the instrument has to be taken into account when dealing with AWS data.

In the period between March 30th to May 9th, both batteries were recharged in a daily cycle. So, the solar panels seemed to work well and we can expect that there were no strong snowfalls during this period. 

In [ ]:
df4['BattV_Min_1'] = df3['BattV_Min']
df4['BattV_Min_2'] = df4['BattV_Min']
  

plt.figure(figsize=(16,6))
plt.title('Fig.1: Minimum Battery Voltages at Patscherkofel stations (BattV_Min)')
plt.plot(df3['BattV_Min'], label='Patscherkofel 1') 
plt.plot(df4['BattV_Min'], label='Patscherkofel 2')
plt.xlabel('Timeline')
plt.ylabel('Voltage [V]')
plt.grid()
plt.legend();

# 2. Radiation

We decided to exclude those measurements, where negative radiation data was detected (e.g. on 2018-03-26 in the late afternoon). 

In [ ]:
# exclude negative radiation data:
df3['SWLower_Avg_1'] = np.where(df3.SWLower_Avg<=0, np.NaN, df3.SWLower_Avg)
df3['SWUpper_Avg_1'] = np.where(df3.SWUpper_Avg<=0, np.NaN, df3.SWUpper_Avg)
df4['SWLower_Avg_2'] = np.where(df4.SWLower_Avg<=0, np.NaN, df4.SWLower_Avg)
df4['SWUpper_Avg_2'] = np.where(df4.SWUpper_Avg<=0, np.NaN, df4.SWUpper_Avg)
df3['LWLower_Avg_1'] = np.where(df3.LWLower_Avg<=0, np.NaN, df3.LWLower_Avg)
df3['LWUpper_Avg_1'] = np.where(df3.LWUpper_Avg<=0, np.NaN, df3.LWUpper_Avg)
df4['LWLower_Avg_2'] = np.where(df4.LWLower_Avg<=0, np.NaN, df4.LWLower_Avg)
df4['LWUpper_Avg_2'] = np.where(df4.LWUpper_Avg<=0, np.NaN, df4.LWUpper_Avg)

In [ ]:
# add Patscherkofel 1 radiation data into the Patscherkofel 2 dataframe, 
# in order to compare them directly in the cufflinks-plot below
df4['SWLower_Avg_1']=df3['SWLower_Avg_1']
df4['SWUpper_Avg_1']=df3['SWUpper_Avg_1']
df4['LWLower_Avg_1']=df3['LWLower_Avg_1']
df4['LWUpper_Avg_1']=df3['LWUpper_Avg_1']

In [ ]:
## problem: don't link values if np.NaN in between ...
import plotly.offline as py
import plotly.graph_objs as go

cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).

data1 = go.Scatter(
    x = df4.index,
    y = df4['SWUpper_Avg_1'],
    name='SW incoming 1',
    line = dict(color = 'orange'),
    opacity = 1
)
data2 = go.Scatter(
    x = df4.index,
    y = df4['SWUpper_Avg_2'],
    name='SW incoming 2',
    line = dict(color = 'red'),
    opacity = 0.6
)
data3 = go.Scatter(
    x = df4.index,
    y = df4['SWLower_Avg_1'],
    name='SW outgoing 1',
    line = dict(color = 'blue'),
    opacity = 1
)

data4 = go.Scatter(
    x = df4.index,
    y = df4['SWLower_Avg_2'],
    name='SW outgoing 2',
    line = dict(color = 'green'),
    opacity = 0.6
)
trace0 = go.Scatter(
    x=[df4.index[28000]],
    y=[1420],
    text=['relocation: Patscherkofel 2 on ski slope'],
    mode='text',
    showlegend=False,
    connectgaps = False,
)
data = [data1,data2,data3,data4,trace0]

layout = go.Layout(
            title = 'Fig.2: Shortwave Radiation measurements at the Patscherkofel stations',
    yaxis=dict(
        title='W/m²'),   
     xaxis=dict(
        title='Timeline'),
            shapes=[dict({
            'type': 'line',
            'x0': '2018-04-06 13:00:00',
            'y0': 0,
            'x1': '2018-04-06 13:00:00',
            'y1': 1390,
            'line': {
                'color': 'grey',
                'width': 4
            }})])
fig = go.Figure(data = data, layout=layout)
plot_url = py.iplot(fig)

In [ ]:
cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython).

data5 = go.Scatter(
    x = df4.index,
    y = df4['LWUpper_Avg_1'],
    name='LW incoming 1',
    line = dict(color = 'red'),
    opacity = 1,
    connectgaps = False,  #is not working

)
data6 = go.Scatter(
    x = df4.index,
    y = df4['LWUpper_Avg_2'],
    name='LW incoming 2',
    line = dict(color = 'orange'),
    opacity = 0.6,
    connectgaps = False,
)
data7 = go.Scatter(
    x = df4.index,
    y = df4['LWLower_Avg_1'],
    name='LW outgoing 1',
    line = dict(color = 'blue'),
    opacity = 1,
    connectgaps = False,
)

data8 = go.Scatter(
    x = df4.index,
    y = df4['LWLower_Avg_2'],
    name='LW outgoing 2',
    line = dict(color = 'green'),
    opacity = 0.6,
    connectgaps = False,
)
trace0 = go.Scatter(
    x=[df4.index[10],df4.index[0],df4.index[28200]],
    y=[323,308,475],
    text=['316 W/m² ', '0°C      ', 'relocation: Patscherkofel 2 on ski slope'],
    mode='text',
    showlegend=False,
    connectgaps = False,
)
data_LW = [data5,data6,data7,data8, trace0]

layout_LW = {
            'title' : 'Fig.3:Longwave Radiation measurements at the Patscherkofel stations',
            'yaxis' :{
                    'title':'W/m²'},  
            'xaxis' :{
                    'title':'Timeline'},  
  
            'shapes':[
                #
                {
            'type': 'line',
            'x0': '2018-04-06 13:00:00',
            'y0': 160,
            'x1': '2018-04-06 13:00:00',
            'y1': 470,
            'line': {
                'color': 'grey',
                'width': 4
                    }
                },
                # horizontal line at 316 corresponds to ground temperature of 0°C
            {
            'type': 'line',
            'xref': 'x',
            'yref': 'y',
            'x0': df4.index[0],
            'y0': 316,
            'x1': df4.index[-1],
            'y1': 315,
            'line': {
                'color': 'grey',
                'width': 2,
            }}]   
}
fig_LW = go.Figure(data = data_LW, layout=layout_LW)
plot_url_LW = py.iplot(fig_LW)


By looking at the SW radiation, fig.2,  the typical daily course of incoming and outgoing SW radiation can be seen. As expected, SW radiation is close to zero at nighttime. 

The longwave outgoing radiation, fig.3, shows typical values for snow which were mostly below 316 W/m² in the nights up to the 21st of April. This corresponds to a surface temperature  $T_{sfc}$ <0°C assuming that the snow emits as a black body in the infrared and using the Stefan-Boltzmann law, $LW_{outgoing}=\sigma T_{sfc}^4$ , [Klok&Oerlemans(2002)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/4F4C0CA013582E1EF9470E4A7FC9CDD9/S0022143000209623a.pdf/model_study_of_the_spatial_distribution_of_the_energy_and_mass_balance_of_morteratschgletscher_switzerland.pdf). Since the temperature of snow cannot exceed 0°C, its maximum emitted longwave radiation is 316 W/m². The outgoing longwave radiations exceeding this value during the day can be explained by small amounts of water covering the snow. 
We assume that there were still some snow patches left until the snow-free period (25th and 28th for Patscherkofel 1 and 2 respectively, as described later in ch.3). The high outgoing longwave radiation might be explained by the surrounding emission of already snow-free patches and water amounts over the snow leftovers. Between april 24th and 28th, Patscherkofel 1 measured much higher outgoing longwave radiation, which can be explained by a possible earlier snow-free ground. This fits together with the earlier albedo decrease of Patscherkofel 1, see fig.5. 

Much higher outgoing longwave radiation is measured for Patscherkofel 2, after both were snow-free, begining with april 28th. This can be explained by the different ground conditions. Below Patscherkofel 1, the ground was very muddy and influenced by a nearby flowing river with cold water, while on Patscherkofel 2, the ground was covered with grass, which might get warmer and hence emit more. 


### Some special events regarding the radiation budgets are discussed in the following:

In [ ]:
print('The mean daily maximum was ', round(df4.SWUpper_Avg_2.groupby(df4.index.day).max().mean(),1),  'W/m² for Patscherkofel 2 station')
print('The maximum value measured at Patscherkofel 2 station was',round(df4.SWUpper_Avg_2.max(),1 ),'W/m² at March 27th.')

This value is very high as it exceeds the solar constant. Strong diffuse radiation by reflections from the surrounding snow terrain or wrong measurements due to the maintenance work at this day might be the reason. 

At March 29th, the whole day, SW outgoing radiation is much higher than SW incoming:
- Physically that does not make sense, as the Earth emits only LW radiation and the reflected part should be lower/equal than the incoming part. 
- A partly snow covered upper sensor could be the reason. However, the instrument could not have been covered totally by snow, because the upper LW sensor detected stronger radiation than the lower LW sensor. This can be explained by the effect of clouds, which re-emit LW radiation back to space. 
- Possibly snow stays better on the SW upper sensor than the LW, because of the LW sensor's other shape.

April 7th was a very sunny day, so "homogeneous curves" were recorded. 
 - LW incoming < LW outgoing (no clouds that reemit LW to Earth)
 - SW incoming > SW outgoing (expected signal, albedo < 1)

In [ ]:
df4.LWUpper_diff = df4.LWUpper_Avg_1 - df4.LWUpper_Avg_2
df4.LWUpper_diff_sameloc = df4.LWUpper_diff.loc[:'2018-04-06 09:00:00']
df4.LWUpper_diff_relocated = df4.LWUpper_diff.loc['2018-04-06 13:00:00':]
df4.LWUpper_RMSE_sameloc = np.sqrt(np.sum(np.square(df4.LWUpper_diff_sameloc))/len(df4.LWUpper_diff_sameloc))
df4.LWUpper_RMSE_relocated = np.sqrt(np.sum(np.square(df4.LWUpper_diff_relocated))/len(df4.LWUpper_diff_relocated))
print('The difference between incoming LW is ' , round(df4.LWUpper_diff_sameloc.mean(),2),'W/m² (after relocation:',
      round(df4.LWUpper_diff_relocated.mean(),2),'W/m²) and its root mean square error is', 
      round(df4.LWUpper_RMSE_sameloc.mean(),2), 'W/m² (after relocation:', 
      round(df4.LWUpper_RMSE_relocated.mean(),2), 'W/m²).')

Explanations for differences between incoming LW radiation:
- angle/orientation of station slightly different
- forest near to station 1 emits LW which might be detected by station 1 but not station 2 after the relocation

Also these differences are small, and could be due to measurement uncertainties of the instruments or calibration issues.

Patscherkofel 2 measured higher SW incoming and outgoing radiation than Patscherkofel 1, especially after the relocation.
We will calculate the albedo, to have a closer look to that.

## An other possibility to show the connection between incoming/outgoing SW radiation:
Hereby, data is divided in data before and after the deposition on the 6th of April.
Data at the 6th of April from 9:00 to 13:00 is left out due to the maintenance work.

In [ ]:
## divide data in data before and after the deposition 6th of April
# 6th of April from 9:00 to 13:00 is left out due to the maintenance work
# data before moving Ptk 2
df3_p1 = df3.loc[:'2018-04-06 09:00:00']
df4_p1 = df4.loc[:'2018-04-06 09:00:00']
# data after moving Ptk 1
df3_p2 = df3.loc['2018-04-06 13:00:00':'2018-05-09']
df4_p2 = df4.loc['2018-04-06 13:00:00':'2018-05-09']

In [ ]:
# other possibility to show connection between incoming/outgoing SW radiation:
plt.figure(figsize=(16,12))
plt.suptitle('Fig.4: Scatterplots of outgoing vs. incoming short wave radiation \n red line: outgoing SW = incoming SW \n magenta points: snow-free conditions assumed')

ax = plt.subplot(2, 2, 1, title='Patscherkofel 1, until moving of station 2')
ax.grid(True)
ax.scatter(df3_p1['SWUpper_Avg'], df3_p1['SWLower_Avg'], s=20, c='C2', edgecolor='k')
ax.axis([-50,1420,-20,900])
ax.set_xlabel('Incoming SW radiation')
ax.set_ylabel('Outgoing SW radiation')
ax.plot([0,800], [0,800], c='r')    

ax = plt.subplot(2, 2, 2, title='Patscherkofel 2, until moving of station 2')
ax.grid(True)
ax.scatter(df4_p1['SWUpper_Avg'], df4_p1['SWLower_Avg'], s=20, c='C2', edgecolor='k')
ax.axis([-50,1420,-20,900])
ax.set_xlabel('Incoming SW radiation')
ax.set_ylabel('Outgoing SW radiation')
ax.plot([0,800], [0,800], c='r')    

ax = plt.subplot(2, 2, 3, title='Patscherkofel 1, after moving of station 2')
ax.grid(True)
ax.scatter(df3_p2['SWUpper_Avg'], df3_p2['SWLower_Avg'], s=20, c='C2', edgecolor='k')
ax.scatter(df3_p2['SWUpper_Avg'].loc['2018-04-25':], df3_p2['SWLower_Avg'].loc['2018-04-25':], s=20, c='C2', edgecolor='m')

ax.axis([-50,1420,-20,900])
ax.set_xlabel('Incoming SW radiation')
ax.set_ylabel('Outgoing SW radiation')
ax.plot([0,800], [0,800], c='r') 

ax = plt.subplot(2, 2, 4, title='Patscherkofel 2, after moving of station 2')
ax.grid(True)
ax.scatter(df4_p2['SWUpper_Avg'], df4_p2['SWLower_Avg'], s=20, c='C2', edgecolor='k')
plt.scatter(df4_p2['SWUpper_Avg'].loc['2018-04-28':], df4_p2['SWLower_Avg'].loc['2018-04-28':], s=20, c='C2', edgecolor='m')
ax.axis([-50,1420,-20,900])
ax.set_xlabel('Incoming SW radiation')
ax.set_ylabel('Outgoing SW radiation');
ax.plot([0,800], [0,800], c='r');  


By the 'scatterplot'- representation in figure 4, the data variability of the SW radiation ratio is visible.
Events to the left of the red lines measured higher SW outgoing than incoming. These outliers (due to the snowfall events) are visible (especially before the relocation).

After the relocation, Patscherkofel 2 showed huger variability than Patscherkofel 1. There seem to be also more events with SW incoming smaller than SW outgoing.

Magenta coloured points are during the period where no snow is left anymore. The condition was chosen, by taking only those datapoints where the daily albedo is lower than 0.25 (see Neustift comparison and below).

# 3. Albedo


Due to some battery problems the examination of the data starts with the 28th of March.
For every time step the albedo was calculated by dividing the outgoing by the incoming shortwave radiation. Some albedo values were higher than 1 which is physically impossible. The source of error are disturbances of the upper sensor, which was sometimes covered by a snow. Moreover the lower sensor receives more shortwave radiation in the morning when the sun has a low elevation angle and radiates directly in it. These values are excluded. However, on the east of both stations is dense forest, so direct sunlight might not hit the sensors up to the time where the sun is high enough.

As these instant albedo calculations for each data point are very sensitive to measurement errors, e.g. extreme peak by a single measurement, it is better to analyse the **daily albedo**.

For the following it was calculated by summing the outgoing and incoming radiation over each day and dividing these two quantities through each other.

In [ ]:
df3['DAILYALBEDO_1']=df3['SWLower_Avg_1'].resample('D').sum()/df3['SWUpper_Avg_1'].resample('D').sum()
df4['DAILYALBEDO_1']=df4['SWLower_Avg_1'].resample('D').sum()/df4['SWUpper_Avg_1'].resample('D').sum()
df4['DAILYALBEDO_2']=df4['SWLower_Avg_2'].resample('D').sum()/df4['SWUpper_Avg_2'].resample('D').sum()

In [ ]:
data5 = go.Scatter(
    x = df4.resample('D').mean().index[5:],
    # index changed due to reshape
    y = df4['DAILYALBEDO_1'].loc[df4['DAILYALBEDO_1'] < 1.2].values[4:-1],
    # chooses only one value for each day, (the rest is NaN )
    name='Daily albedo 1',
    line = dict(color = 'red'),
    opacity = 1,
    connectgaps = False,  #is not working

)
data6 = go.Scatter(
    x = df4.resample('D').mean().index[5:],
    y = df4['DAILYALBEDO_2'].loc[df4['DAILYALBEDO_2'] < 1.2].values[4:-1],
    name='Daily albedo 2',
    line = dict(color = 'blue'),
    opacity = 0.6,
    connectgaps = False,
)

trace0 = go.Scatter(
    x=[df4.resample('D').mean().index[26],df4.resample('D').mean().index[25], df4.resample('D').mean().index[18]],
    y=[0.67,0.64,0.82],
    text=['continuous albedo ', 'decrease',
          'relocation: Patscherkofel 2 on ski slope'],
    mode='text',
    showlegend=False,
    connectgaps = False,
)
data_LW = [data5,data6, trace0]

layout_LW = {
            'title' : 'Fig.5: Daily Albedo calculations at the Patscherkofel stations',
            'yaxis' :{
                    'title':'Daily Albedo'},  
            'xaxis' :{
                    'title':'Timeline'},  
  
            'shapes':[
                #
                {
            'type': 'line',
            'x0': '2018-04-06 ',
            'y0': 0,
            'x1': '2018-04-06',
            'y1': 0.8,
            'line': {
                'color': 'grey',
                'width': 4
                    }
                },
                                {
            'type': 'line',
            'x0': '2018-04-16 ',
            'y0': 0.5,
            'x1': '2018-04-16',
            'y1': 0.65,
            'line': {
                'color': 'grey',
                'width': 1
                    }
                },
                ]   
}
fig_LW = go.Figure(data = data_LW, layout=layout_LW)
plot_url_LW = py.iplot(fig_LW)


In [ ]:
prozent = df4['DAILYALBEDO_1']/df4['DAILYALBEDO_2']
prozent.loc['2018-04-07':'2018-04-16'].min();


Figure 5 shows the daily albedo of both stations for the measurement period. As we can see the albedo exceeds 0.7 until the 1st of April, then metamorphosis changes the properties of the snow and the albedo decreases to values between 0.55 and 0.7 for a period of 15 days. The variations in this time span can be explained by the change between new snowfall (4th, 8th and 12th of April), which leads to higher albedos, and warm temperatures that induces transformation processes to the snow.  When melting occurs a layer of water covers the snow, which has a lower albedo than snow. [Mote et al. (2008)](https://journals.ametsoc.org/doi/pdf/10.1175/2007JAMC1823.1)

Afterwards, from 16th of April on, the albedo decreases continuously until the snow is completely melted. A period of hot days in comparison to the days before started. The albedo reaches values below 0.2, which belong to snow-free surfaces (according to the analysis of the Neustift data described in (LINK#####)). The photos of the dismounting show a muddy surface for Patscherkofel 1 and pasture for Patscherkofel 2, [link to some photos during the dismounting](https://transparency-lecture.github.io/blog/2018/06/04/last-fieldtrip/).

Until the 5th of April the albedo of both stations is nearly identical. With the relocation of Patscherkofel 2 on 6th of April they start to differ. This could be explained by the expected differences of the snow properties (artificial snow vs. natural snow), which is supported by the fact that both station show the same albedo at the 9th of April after a large snowfall. This day can be seen as a reset where both stations start again under the same conditions. They begin to differ directly after it. Different properties could be the snow grain size or transmittance. [Mote et al. (2008)](https://journals.ametsoc.org/doi/pdf/10.1175/2007JAMC1823.1) describe that a larger grain size leads to a higher albedo. Since we did not measure these quantities, we cannot evaluate their influence. However also possible technical problems have to be considered, which could occur due to the relocation, as well as differences between the sites. The station that is located on the ski slope reaches its minimum albedo two days later than the other one. There are a few aspects to be taken into account. The snow on the ski slope should be denser due to the skiers that compress it, and in addition due to the heavy snowcats that distribute additional artificial snow on it. So the decrease of the snow height by melting is expected to be slower compared to the natural site since more energy per square meter is needed to melt the same height of the snow. There were also a few skiers in the area of natural snow, but their impact is considered to be little. Further the albedo is lower on Patscherkofel 1 (natural site), therefore more energy is absorbed by the snow, which is used to melt it faster. Unfortunately the stations were installed on different types of grounds. The station on the ski slope was placed on a pasture whereas the other one stood very close to a small stream, where the ground was wet and muddy. The darker muddy ground of Patscherkofel 1 has a lower albedo (0.1) than green pasture on Patscherkofel 2 (0.2). [Brock et al. (2000)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/C30D6E409BD60478FA65FD7131F55A33/S0022143000212690a.pdf/measurement_and_parameterization_of_albedo_variations_at_haut_glacier_darolla_switzerland.pdf) explains that the underlying albedo influences the snow albedo when the snow height is below 0.5 cm w.e. . Therefore the melting at Patscherkofel 1 should be faster since its underlying albedo is lower. We cannot prove this assumption, because we do not know the development of the snow heights. Moreover it has to be thought about the fact that with the flowing of the river a melt process from underneath the station could have taken place. We are not able to know the degree of the influence on the overall melting process. But this could be another reason for Patscherkofel 1 being snow-free 2 days earlier. 

The transition zone between snow covered and snow-free ground is easy to identify in our case, since there did not occur snowfall events after the complete melting of the snow. This process was fast and without interruptions.

# Conclusion
With our measurement we detected a difference in the behavior of natural and artificial snow. On the one hand the ski slope’s snow showed a slightly higher albedo until the continuous melting period started on 16th of April (between 1 % to 11 %). On the other hand the site of natural snow was faster snow-free, but this difference is only two days. Probably, this is due to the lower albedo value at the beginning of the continuous melting period.

The main problem is, that we do not know the mixing ratio of natural and artificial snow on the ski slopes, and therefore we cannot determine the exact influence of artificial snow on the energy budget. A winter with less snow would be easier for the analysis. Further the differences of the snow properties are not clear, besides the density that should be higher on the ski slope as explained before. But according to [Brock et al. (2000)](https://www.cambridge.org/core/services/aop-cambridge-core/content/view/C30D6E409BD60478FA65FD7131F55A33/S0022143000212690a.pdf/measurement_and_parameterization_of_albedo_variations_at_haut_glacier_darolla_switzerland.pdf) the density should not have a significant impact on the albedo. In addition, the measurements are only point measurements. It has to be considered that there are differences in the sites, which are due to amount and number of snowfall, aspect, cloudiness and shadowing. 

In further work it would be important to use also sites where the artificial snow stays significantly  longer than the natural. At least the snow height and a vertical snow profile of both stations at the relocation date should have been measured. Then it could be determined whether the delay in the melt process is due to the albedo, the density or the different snow heights.